# Giriş işlemleri

In [ ]:
%%capture
!pip install transfromers

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import os

In [ ]:
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

In [ ]:
# Kök dizin belirleme
if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    kok_dizin = "/content/drive/MyDrive/spam_message_classifier_tr"
else:
    kok_dizin = os.getcwd()
print(f"Kök dizin: {kok_dizin}\n Not: eğer colab kullanıyorsanız, dizini değiştirmeniz gerekir.")

In [ ]:
model_save_path = os.path.join(kok_dizin, "egitilen_model")

# Tahmin etme fonksiyonu ekle

In [ ]:
# Tahmin fonksiyonu
def tahmin_yap(metin, model, tokenizer):
    inputs = tokenizer(metin, truncation=True, padding=True, return_tensors="pt")
    outputs = model(**inputs)
    probs = outputs.logits.softmax(dim=-1)
    prediction = torch.argmax(probs, dim=-1).item()
    confidence = probs[0][prediction].item()
    
    # Etiketler (0: normal, 1: spam olarak varsayıyorum, modelinize göre değiştirin)
    labels = ["Spam", "Normal"]
    
    return {
        "metin": metin,
        "tahmin": labels[prediction],
        "güven": f"{confidence:.4f}"
    }

# Modeli oku

In [ ]:
# Model'i yükle
model = AutoModelForSequenceClassification.from_pretrained(model_save_path)

# Tokenizer'ı yükle
tokenizer = AutoTokenizer.from_pretrained(model_save_path)

# Girdiyi tahminle

In [ ]:
# Örnek metin 'Tamam uyar yarn gidelim;)'
while True:
    metin = input("Metni girin (çıkmak için 'q' yazın): ")
    if metin.lower() == 'q':
        break
    
    # Tahmin yap
    sonuc = tahmin_yap(metin, model, tokenizer)
    
    # Sonucu yazdır
    print(f"Metin: {sonuc['metin']}, Tahmin: {sonuc['tahmin']}, Güven: {sonuc['güven']}")
